In [390]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling as pp

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt


In [391]:
from sklearn import preprocessing

In [392]:
sample_sub_csv = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
train_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [393]:
flag = 0 
for column in test_df.columns:
    if(column == "SalePrice"):
        flag = 1
if(flag == 1):
    print("There is SalePrice beware ")
else:
    print("You need to predict SalePrice")

You need to predict SalePrice


In [394]:
# Because we want to predict the SalePrice
Y_train = train_df['SalePrice']
Y_train.shape

(1460,)

In [395]:
# We have not removed the SalePrice to analyse the Missing Values and do class-conditions Regression
print("The number of columns before making the augmentation = ",train_df.shape[1])
train_df.drop(['Alley', 'Exterior1st', 'Exterior2nd', 'FireplaceQu', 'Fence', 'MiscFeature'],axis = 1, inplace = True)
train_df.drop(['PoolQC', 'Utilities', '3SsnPorch', 'BsmtFinSF2', 'PoolArea', 'Street','MiscVal'],axis = 1, inplace = True)
train_df.drop(['EnclosedPorch','ScreenPorch','LotShape','KitchenAbvGr','GarageQual','LowQualFinSF','RoofMatl','Heating','OpenPorchSF','MasVnrArea','LandSlope'], axis = 1, inplace = True)
train_df.drop(['GarageCond','BsmtHalfBath','BsmtFinType2','GarageCars','Condition2','WoodDeckSF'],axis = 1, inplace = True)
print("The number of columns after dropping the missing valued columns and the predicting variable = ",train_df.shape[1])

The number of columns before making the augmentation =  81
The number of columns after dropping the missing valued columns and the predicting variable =  51


In [396]:
print("The number of columns before making augmenation in Test set",test_df.shape[1])
test_df.drop(['Alley', 'Exterior1st', 'Exterior2nd', 'FireplaceQu', 'Fence', 'MiscFeature'],axis = 1, inplace = True)
test_df.drop(['PoolQC', 'Utilities', '3SsnPorch', 'BsmtFinSF2', 'PoolArea', 'Street','MiscVal'],axis = 1, inplace = True)
test_df.drop(['EnclosedPorch','ScreenPorch','LotShape','KitchenAbvGr','GarageQual','LowQualFinSF','RoofMatl','Heating','OpenPorchSF','MasVnrArea','LandSlope'], axis = 1, inplace = True)
test_df.drop(['GarageCond','BsmtHalfBath','BsmtFinType2','GarageCars','Condition2','WoodDeckSF'],axis = 1, inplace = True)
print("The number of columns after dropping the missing valued columns = ",test_df.shape[1])

The number of columns before making augmenation in Test set 80
The number of columns after dropping the missing valued columns =  50


In [397]:
def check_mismatch(column, train_df, test_df):
    if(train_df[column].isnull().any()):
        print("Number of rows haivng the NUll value train_df = ",train_df[column].isnull().sum())
    else:
        print("There is no Null value in train_df column ",column)
    
    if(test_df[column].isnull().any()):
        print("Number of rows haivng the NUll value in test_df = ",test_df[column].isnull().sum())
    else:
        print("There is no Null value in train_df column ",column)    

In [398]:
def check_missing(column, check_df):
    if(check_df[column].isnull().any()):
        print("Number of rows having the NULL value in ",check_df," are ",check_df[column].isnull().sum())
    else:
        print("There is no NULL vale in the column", column)

In [399]:
train_df.drop('SalePrice',axis = 1, inplace = True)

In [400]:
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,LandContour,LotConfig,Neighborhood,Condition1,BldgType,...,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageArea,PavedDrive,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Lvl,Inside,NAmes,Feedr,1Fam,...,0,Attchd,1961.0,Unf,730.0,Y,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Lvl,Corner,NAmes,Norm,1Fam,...,0,Attchd,1958.0,Unf,312.0,Y,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Lvl,Inside,Gilbert,Norm,1Fam,...,1,Attchd,1997.0,Fin,482.0,Y,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Lvl,Inside,Gilbert,Norm,1Fam,...,1,Attchd,1998.0,Fin,470.0,Y,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,HLS,Inside,StoneBr,Norm,TwnhsE,...,0,Attchd,1992.0,RFn,506.0,Y,1,2010,WD,Normal


In [401]:
test_df.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,LandContour,LotConfig,Neighborhood,Condition1,BldgType,...,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageArea,PavedDrive,MoSold,YrSold,SaleType,SaleCondition
1454,2915,160,RM,21.0,1936,Lvl,Inside,MeadowV,Norm,Twnhs,...,0,NaN,NaN,NaN,0.0,Y,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Lvl,Inside,MeadowV,Norm,TwnhsE,...,0,CarPort,1970.0,Unf,286.0,Y,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Lvl,Inside,Mitchel,Norm,1Fam,...,1,Detchd,1960.0,Unf,576.0,Y,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Lvl,Inside,Mitchel,Norm,1Fam,...,0,NaN,NaN,NaN,0.0,Y,7,2006,WD,Normal
1458,2919,60,RL,74.0,9627,Lvl,Inside,Mitchel,Norm,1Fam,...,1,Attchd,1993.0,Fin,650.0,Y,11,2006,WD,Normal


In [402]:
train_ending = 1460       # This is the Id numbeer
train_starting = 1
test_starting = 1461
test_ending = 2919

In [403]:
# Merge the two datafames to enocde them properly
train_df.set_index('Id', inplace = True)
test_df.set_index('Id', inplace = True)

In [404]:
frames = [train_df, test_df]
combined_df = pd.concat(frames)

In [405]:
combined_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,LandContour,LotConfig,Neighborhood,Condition1,BldgType,HouseStyle,...,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageArea,PavedDrive,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Lvl,Inside,CollgCr,Norm,1Fam,2Story,...,0,Attchd,2003.0,RFn,548.0,Y,2,2008,WD,Normal
2,20,RL,80.0,9600,Lvl,FR2,Veenker,Feedr,1Fam,1Story,...,1,Attchd,1976.0,RFn,460.0,Y,5,2007,WD,Normal
3,60,RL,68.0,11250,Lvl,Inside,CollgCr,Norm,1Fam,2Story,...,1,Attchd,2001.0,RFn,608.0,Y,9,2008,WD,Normal
4,70,RL,60.0,9550,Lvl,Corner,Crawfor,Norm,1Fam,2Story,...,1,Detchd,1998.0,Unf,642.0,Y,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Lvl,FR2,NoRidge,Norm,1Fam,2Story,...,1,Attchd,2000.0,RFn,836.0,Y,12,2008,WD,Normal


In [406]:
combined_df.tail()

,MSSubClass,MSZoning,LotFrontage,LotArea,LandContour,LotConfig,Neighborhood,Condition1,BldgType,HouseStyle,...,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageArea,PavedDrive,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
2915,160,RM,21.0,1936,Lvl,Inside,MeadowV,Norm,Twnhs,2Story,...,0,NaN,NaN,NaN,0.0,Y,6,2006,WD,Normal
2916,160,RM,21.0,1894,Lvl,Inside,MeadowV,Norm,TwnhsE,2Story,...,0,CarPort,1970.0,Unf,286.0,Y,4,2006,WD,Abnorml
2917,20,RL,160.0,20000,Lvl,Inside,Mitchel,Norm,1Fam,1Story,...,1,Detchd,1960.0,Unf,576.0,Y,9,2006,WD,Abnorml
2918,85,RL,62.0,10441,Lvl,Inside,Mitchel,Norm,1Fam,SFoyer,...,0,NaN,NaN,NaN,0.0,Y,7,2006,WD,Normal
2919,60,RL,74.0,9627,Lvl,Inside,Mitchel,Norm,1Fam,2Story,...,1,Attchd,1993.0,Fin,650.0,Y,11,2006,WD,Normal


In [407]:
def encode_categorical(df, column_list, index_list):
    for column in column_list:
        df[column] = df[column].astype('str')
        encoder = preprocessing.LabelEncoder()
        encoded_list = encoder.fit_transform(df[column])
#         print(encoded_list)
#         print(len(encoded_list))
        encoded_series = pd.Series(encoded_list, index = index_list)
        df[column] = encoded_series
        print("The ", column, "is encoded ")
    return(df)

In [408]:
index_list = []
for i in range(1, 2920):
    index_list.append(i)

In [409]:
#pp.ProfileReport(combined_df)

In [410]:
columns_encoded = 0
categorical_list = ['MSSubClass','MSZoning','BldgType','YearBuilt','YrSold','BsmtFullBath','CentralAir','Condition1','Electrical','ExterCond','YearRemodAdd']
columns_encoded += len(categorical_list)
combined_df = encode_categorical(combined_df, categorical_list, index_list)

print("Number of columns encoded ",columns_encoded)

The  MSSubClass is encoded 
The  MSZoning is encoded 
The  BldgType is encoded 
The  YearBuilt is encoded 
The  YrSold is encoded 
The  BsmtFullBath is encoded 
The  CentralAir is encoded 
The  Condition1 is encoded 
The  Electrical is encoded 
The  ExterCond is encoded 
The  YearRemodAdd is encoded 
Number of columns encoded  11


In [411]:
categorical_list = ['ExterQual','Fireplaces','Foundation','FullBath','Functional','GarageYrBlt','HalfBath','HeatingQC','HouseStyle','KitchenQual','LandContour']
columns_encoded += len(categorical_list)
combined_df = encode_categorical(combined_df, categorical_list, index_list)

print("Number of columns encoded ",columns_encoded)

The  ExterQual is encoded 
The  Fireplaces is encoded 
The  Foundation is encoded 
The  FullBath is encoded 
The  Functional is encoded 
The  GarageYrBlt is encoded 
The  HalfBath is encoded 
The  HeatingQC is encoded 
The  HouseStyle is encoded 
The  KitchenQual is encoded 
The  LandContour is encoded 
Number of columns encoded  22


In [412]:
categorical_list = ['LotConfig','MasVnrType','MoSold','Neighborhood','OverallCond','OverallQual','PavedDrive','RoofStyle','SaleCondition','SaleType','TotRmsAbvGrd']
columns_encoded += len(categorical_list)
combined_df = encode_categorical(combined_df, categorical_list, index_list)

print("Number of columns encoded ",columns_encoded)

The  LotConfig is encoded 
The  MasVnrType is encoded 
The  MoSold is encoded 
The  Neighborhood is encoded 
The  OverallCond is encoded 
The  OverallQual is encoded 
The  PavedDrive is encoded 
The  RoofStyle is encoded 
The  SaleCondition is encoded 
The  SaleType is encoded 
The  TotRmsAbvGrd is encoded 
Number of columns encoded  33


In [413]:
#pp.ProfileReport(combined_df)

In [414]:
combined_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,LandContour,LotConfig,Neighborhood,Condition1,BldgType,HouseStyle,...,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageArea,PavedDrive,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,10,3,65.0,8450,3,4,5,2,0,5,...,0,Attchd,94,RFn,548.0,2,4,2,8,4
2,5,3,80.0,9600,3,2,24,1,0,2,...,1,Attchd,67,RFn,460.0,2,7,1,8,4
3,10,3,68.0,11250,3,4,5,2,0,5,...,1,Attchd,92,RFn,608.0,2,11,2,8,4
4,11,3,60.0,9550,3,0,6,2,0,5,...,1,Detchd,89,Unf,642.0,2,4,0,8,0
5,10,3,84.0,14260,3,2,15,2,0,5,...,1,Attchd,91,RFn,836.0,2,3,2,8,4


In [415]:
combined_df.tail()

,MSSubClass,MSZoning,LotFrontage,LotArea,LandContour,LotConfig,Neighborhood,Condition1,BldgType,HouseStyle,...,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageArea,PavedDrive,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
2915,2,4,21.0,1936,3,4,10,2,3,5,...,0,NaN,103,NaN,0.0,2,8,0,8,4
2916,2,4,21.0,1894,3,4,10,2,4,5,...,0,CarPort,61,Unf,286.0,2,6,0,8,0
2917,5,3,160.0,20000,3,4,11,2,0,2,...,1,Detchd,51,Unf,576.0,2,11,0,8,0
2918,14,3,62.0,10441,3,4,11,2,0,6,...,0,NaN,103,NaN,0.0,2,9,0,8,4
2919,10,3,74.0,9627,3,4,11,2,0,5,...,1,Attchd,84,Fin,650.0,2,2,0,8,4


In [416]:
# Numeric types need to do MinxMaxScaler
def scale_data(df, column_list, index_list):
    for column in column_list:
        df[column] = df[column].astype('float')
        encoder = preprocessing.StandardScaler()
        df[column] = encoder.fit_transform(df[column].values.reshape(-1,1))
        print("The ",column, "is encoded")
    return(df)

In [417]:
scale_column_list = ['BsmtUnfSF', '2ndFlrSF', 'BsmtFinSF1', 'GarageArea', 'GrLivArea', 'LotArea', 'TotalBsmtSF','1stFlrSF']
columns_encoded += len(scale_column_list)
combined_df = scale_data(combined_df, scale_column_list, index_list)

print("Number of columns encoded ",columns_encoded)

The  BsmtUnfSF is encoded
The  2ndFlrSF is encoded
The  BsmtFinSF1 is encoded
The  GarageArea is encoded
The  GrLivArea is encoded
The  LotArea is encoded
The  TotalBsmtSF is encoded
The  1stFlrSF is encoded
Number of columns encoded  41


In [418]:
combined_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,LandContour,LotConfig,Neighborhood,Condition1,BldgType,HouseStyle,...,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageArea,PavedDrive,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,10,3,65.0,-0.217879,3,4,5,2,0,5,...,0,Attchd,94,RFn,0.348840,2,4,2,8,4
2,5,3,80.0,-0.072044,3,2,24,1,0,2,...,1,Attchd,67,RFn,-0.059782,2,7,1,8,4
3,10,3,68.0,0.137197,3,4,5,2,0,5,...,1,Attchd,92,RFn,0.627446,2,11,2,8,4
4,11,3,60.0,-0.078385,3,0,6,2,0,5,...,1,Detchd,89,Unf,0.785323,2,4,0,8,0
5,10,3,84.0,0.518903,3,2,15,2,0,5,...,1,Attchd,91,RFn,1.686149,2,3,2,8,4


In [419]:
combined_df.tail()

,MSSubClass,MSZoning,LotFrontage,LotArea,LandContour,LotConfig,Neighborhood,Condition1,BldgType,HouseStyle,...,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageArea,PavedDrive,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
2915,2,4,21.0,-1.043937,3,4,10,2,3,5,...,0,NaN,103,NaN,-2.195761,2,8,0,8,4
2916,2,4,21.0,-1.049263,3,4,10,2,4,5,...,0,CarPort,61,Unf,-0.867740,2,6,0,8,0
2917,5,3,160.0,1.246808,3,4,11,2,0,2,...,1,Detchd,51,Unf,0.478856,2,11,0,8,0
2918,14,3,62.0,0.034605,3,4,11,2,0,6,...,0,NaN,103,NaN,-2.195761,2,9,0,8,4
2919,10,3,74.0,-0.068620,3,4,11,2,0,5,...,1,Attchd,84,Fin,0.822470,2,2,0,8,4


In [420]:
combined_df['GarageType'] = combined_df['GarageType'].fillna(value = 'None')

In [421]:
combined_df['GarageFinish'] = combined_df['GarageFinish'].fillna(value = 'None')
combined_df['BsmtQual'] = combined_df['BsmtQual'].fillna(value = 'None')
combined_df['BsmtFinType1'] = combined_df['BsmtFinType1'].fillna(value = 'None')
combined_df['BsmtExposure'] = combined_df['BsmtExposure'].fillna(value = 'None')
combined_df['BsmtCond'] = combined_df['BsmtCond'].fillna(value = 'None')

In [422]:
categorical_list = ['GarageType','GarageFinish', 'BsmtQual', 'BsmtFinType1', 'BsmtExposure','BsmtCond']
columns_encoded += len(categorical_list)
combined_df = encode_categorical(combined_df, categorical_list, index_list)
print(columns_encoded)

The  GarageType is encoded 
The  GarageFinish is encoded 
The  BsmtQual is encoded 
The  BsmtFinType1 is encoded 
The  BsmtExposure is encoded 
The  BsmtCond is encoded 
47


In [423]:
combined_df['LotFrontage'] = combined_df['LotFrontage'].fillna(value = 0)
scale_list = ['LotFrontage']
columns_encoded += len(scale_list)
combined_df = scale_data(combined_df, scale_list, index_list)
print(columns_encoded)

The  LotFrontage is encoded
48


In [424]:
combined_df.tail()

,MSSubClass,MSZoning,LotFrontage,LotArea,LandContour,LotConfig,Neighborhood,Condition1,BldgType,HouseStyle,...,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageArea,PavedDrive,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
2915,2,4,-1.098304,-1.043937,3,4,10,2,3,5,...,0,6,103,1,-2.195761,2,8,0,8,4
2916,2,4,-1.098304,-1.049263,3,4,10,2,4,5,...,0,4,61,3,-0.867740,2,6,0,8,0
2917,5,3,3.053937,1.246808,3,4,11,2,0,2,...,1,5,51,3,0.478856,2,11,0,8,0
2918,14,3,0.126458,0.034605,3,4,11,2,0,6,...,0,6,103,1,-2.195761,2,9,0,8,4
2919,10,3,0.484925,-0.068620,3,4,11,2,0,5,...,1,1,84,0,0.822470,2,2,0,8,4


In [425]:
combined_df.shape

(2919, 49)